In [15]:
pip install findspark


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install pyspark findspark

  Using cached pyspark-3.5.3-py2.py3-none-any.whl
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [16]:
import findspark
findspark.init()

In [17]:
spark_url = 'local'

In [18]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

In [19]:
spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark SQL')\
        .getOrCreate()

24/12/07 15:13:19 WARN Utils: Your hostname, Tents-MacBook.local resolves to a loopback address: 127.0.0.1; using 10.201.242.183 instead (on interface en0)
24/12/07 15:13:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/07 15:13:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/07 15:13:20 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


24/12/07 15:13:35 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [20]:
path = '/Users/tentachita/Downloads/DSDE_Final_Project_WARK/Data_Aj/2/output_data_2018.csv'


In [21]:
df= spark.read.csv(path, header=True, inferSchema=True)

In [26]:
df.printSchema()

root
 |-- citation_title: string (nullable = true)
 |-- abstracts: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- affiliations: string (nullable = true)
 |-- classifications: string (nullable = true)
 |-- subject_area_name: string (nullable = true)
 |-- subject_area_code: string (nullable = true)



In [27]:
df.head(1)

[Row(citation_title='Public health and international epidemiology for radiology', abstracts='-', authors='Pongpirul Krit; Lungren Matthew P.', affiliations='Stanford University School of Medicine\\Stanford\\United States; Chulalongkorn University\\Bangkok\\Thailand; Bumrungrad International Hospital\\Bangkok\\Thailand; Stanford Healthcare\\Stanford\\United States; Stanford University\\Palo Alto\\United States; Johns Hopkins Bloomberg School of Public Health\\Baltimore\\United States', classifications='ASJC: 2700; SUBJABBR: MEDI', subject_area_name='Medicine (all)', subject_area_code='2700')]

In [28]:
df = df.drop("affiliations", "subject_area_code")

In [29]:
df.printSchema()

root
 |-- citation_title: string (nullable = true)
 |-- abstracts: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- classifications: string (nullable = true)
 |-- subject_area_name: string (nullable = true)



In [30]:
from pyspark.sql.functions import split

In [40]:
df = df.withColumn("target", split(df["subject_area_name"], ","))
df.show(truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [42]:
from pyspark.ml.feature import VectorAssembler

In [36]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="target", outputCol="label")
df = indexer.fit(df).transform(df)

df.select("target", "label").show(truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|target                                                                                                                                                                             |label|
+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|Medicine (all)                                                                                                                                                                     |0.0  |
|Electrical and Electronic Engineering; Electronic                                                                                                                                  |866.0|
| Optical and Magnetic Materials                            

In [45]:
from pyspark.ml.feature import HashingTF, IDF, Tokenizer

# Tokenize the text
tokenizer = Tokenizer(inputCol="citation_title", outputCol="title_tokens")
df = tokenizer.transform(df)

# Apply HashingTF
hashingTF = HashingTF(inputCol="title_tokens", outputCol="raw_features", numFeatures=100)
df = hashingTF.transform(df)

# Apply IDF
idf = IDF(inputCol="raw_features", outputCol="title_features")
idf_model = idf.fit(df)
df = idf_model.transform(df)

In [47]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=["title_features"], outputCol="features")
df = assembler.transform(df)

# Show the resulting DataFrame
df.select("features", "label").show(truncate=False)


+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----+
|features                                                                                                                                                                                                                                                                                                                                                                                                  |label|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [48]:
df.printSchema()

root
 |-- citation_title: string (nullable = true)
 |-- abstracts: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- classifications: string (nullable = true)
 |-- subject_area_name: string (nullable = true)
 |-- target: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- label: double (nullable = false)
 |-- title_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- raw_features: vector (nullable = true)
 |-- title_features: vector (nullable = true)
 |-- features: vector (nullable = true)



In [49]:
train_df, test_df = df.randomSplit([0.7, 0.3], seed=38)

In [52]:
from pyspark.ml.classification import LogisticRegression

lr = LogisticRegression(featuresCol="features", labelCol="label")
model = lr.fit(train_df)


24/12/07 16:45:05 WARN BlockManager: Asked to remove block broadcast_630, which does not exist


24/12/07 18:31:18 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 464711 ms exceeds timeout 120000 ms
24/12/07 18:31:18 WARN SparkContext: Killing executors is not supported by current scheduler.
24/12/07 18:31:24 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o